In [4]:
import pandas as pd
import numpy as np
import re
import datetime
import string
import nltk
!pip install transformers
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline
from nltk.corpus import stopwords, gutenberg
from nltk.tokenize import RegexpTokenizer, TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.7 MB 2.6 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/9.7 MB 2.5 MB/s eta 0:00:04
   ------ --------------------------------- 1.6/9.7 MB 2.1 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/9.7 MB 2.3 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/9.7 MB 2.3 MB/s eta 0:00:04
   ---------- ----------------------------- 2.6/9.7 MB 1.8 MB/s eta 0:00:04
   ------------- -------------------------- 3.1/9.7 MB 1.9 MB/s eta 0:00:04
   --------------- ------------------------ 3.7/9.7 MB 2.1 MB/s eta 0:00:03
   ----------------- ---------------------- 4.2/9.7 MB 2.0 MB/s eta 0:00:03
   ------------------- -------------------- 4.7/9.7 MB 2.1 MB/s eta 0:00:03
   --------------------- ------------------ 5.2/9.7 MB 2.2 MB/s eta 0:00:03
   --------------------- -

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
df = pd.read_csv('Ethereum_tweets.csv')
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,#1 Crypto Currency TRADING,NaN,"Profit thought trading, Learn , Trade & Earn💲....",2021-11-10 10:42:01+00:00,4,0,0,False,2021-11-15 07:26:39+00:00,Get upto 1000 that’s 10x with our premium spot...,NaN,Twitter for Android,False
1,Reyrey,Kuala Lumpur,Artist | NFT Broker & Collector for AnideaNFT ...,2011-03-07 07:22:57+00:00,343,650,589,False,2021-11-15 07:26:35+00:00,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...","['SupportingCreators', 'CryptoArt', 'CryptoArt...",Twitter for iPhone,False
2,akter jahan,NaN,NaN,2021-10-28 13:28:05+00:00,1,2,0,False,2021-11-15 07:26:32+00:00,Top 10 Coins by Social Engagement over the las...,"['LunarCrush', 'bitcoin', 'dogecoin']",Twitter Web App,False
3,riyan,NaN,i always paricipated your project,2021-09-16 16:59:11+00:00,11,127,99,False,2021-11-15 07:26:30+00:00,Requesting faucet funds into 0x5278942b39deD8c...,"['Rinkeby', 'Ethereum']",Twitter Web App,False
4,Michelle Eriksen,NaN,"Art Creator, NFT, Crypto",2021-08-11 06:40:08+00:00,522,1428,16,False,2021-11-15 07:26:30+00:00,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,"['NFTs', 'nftcollector', 'NFTCommunity', 'art'...",Twitter Web App,False


In [7]:
round(df.isna().sum()/len(df)*100,2)

user_name            0.00
user_location       50.39
user_description    13.72
user_created         0.00
user_followers       0.00
user_friends         0.00
user_favourites      0.00
user_verified        0.00
date                 0.00
text                 0.00
hashtags            24.72
source               0.00
is_retweet           0.00
dtype: float64

In [10]:
df.drop(columns=['user_location', 'user_description', 'user_created', 'user_followers', 'user_friends', 'user_favourites','user_verified', 'source'], inplace=True)

KeyError: "['user_location', 'user_description', 'user_created', 'user_followers', 'user_friends', 'user_favourites', 'user_verified', 'source'] not found in axis"

In [11]:
round(df.isna().sum()/len(df)*100,2)

user_name      0.00
date           0.00
text           0.00
hashtags      24.72
is_retweet     0.00
dtype: float64

In [12]:
nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

classifier = pipeline("sentiment-analysis")

def label_sentiment_vader(tweet):
    scores = analyzer.polarity_scores(tweet)
    compound_score = scores['compound']

    if compound_score > 0.05:
        return 'positive'
    elif compound_score < -0.05:
        return 'negative'
    else:
        return 'neutral'

def label_sentiment_transformer(tweet):
    result = classifier(tweet)[0]
    label = result['label']
    return label.lower()


metode = "transformers"

if metode == "vader":
    df['sentiment'] = df['text'].apply(label_sentiment_vader)
elif metode == "transformers":
    df['sentiment'] = df['text'].apply(label_sentiment_transformer)
else:
    print("Metode pelabelan tidak valid.")

df.to_csv('labeled_tweets.csv', index=False)


[nltk_data] Downloading package vader_lexicon to C:\Users\Rizky
[nltk_data]     Fadhilah\AppData\Roaming\nltk_data...
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\Rizky Fadhilah\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rizky Fadhilah\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.